# Практика 3
Корешков

In [26]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from collections import namedtuple 

def _(*args): return np.array([*args])


```

Задача#1
Знайти розв'язок системи A x =  B методом Гауса, де 

A = {{-3, -8, 3}, {3, -9, -9}, {2, -5, -1}}; B = {5, 7, 0};
A = {{-0.160, 5.603, 8.948}, {-9.494, 1.255, -5.962}, {6.870, -6.467,  2.035}};  B = {9.957, -4.966, 3.032};
виконуючи обчислення з п'ятьма значущими цифрами.

Задача#2
Знайти матрицю обернену до матриці A #1

Задача#3
Визначити число обумовленості матриці A #1
```

In [34]:
PREC = 5

Task = namedtuple('Task', 'A, b')
tasks = [
    Task(A=_(
        [-3, -8, -3],
        [3, -9, -9],
        [2, -5, -1] 
        ), b=_(5, 7, 0)
    ),
    Task(A=_(
        [-0.160, 5.603, 8.948],
        [-9.494, 1.255, -5.962],
        [6.870, -6.467, 2.035]
        ), b=_(9.957, -4.966, 3.032)
    ),
]
tasks

[Task(A=array([[-3, -8, -3],
        [ 3, -9, -9],
        [ 2, -5, -1]]), b=array([5, 7, 0])),
 Task(A=array([[-0.16 ,  5.603,  8.948],
        [-9.494,  1.255, -5.962],
        [ 6.87 , -6.467,  2.035]]), b=array([ 9.957, -4.966,  3.032]))]

## Задача 1

Я не реалізував "обчислення з точністю до певної значущої цифри" в явному вигляді, але використав для обчислень різні типи даних:
-  float16, що має мантісу розміром в 10bit, що еквівалентно $\lfloor\log_{10}(2^{10})\rfloor = 3$ значущим цифрам
-  float32, що має мантісу розміром в 23bit, що еквівалентно $\lfloor\log_{10}(2^{23})\rfloor = 6$ значущим цифрам

In [136]:
def print_aug_mat(A,b, prec=5):
    n,m = A.shape
    out = ""
    colmax = np.abs(A).max(axis=0)
    negs = A.min(axis=0) < 0
    colwidth = (np.maximum(np.log10(colmax), 1) + prec + 3).astype(np.uint8) + negs


    # print(colmax)
    # print(np.log10(colmax))
    # print(colwidth)

    assert n == b.shape[0]
    for i in range(n):
        for j in range(m):
            out += str(np.round(A.astype(np.float64)[i,j], prec)).ljust(colwidth[j])
        out += f" | {np.round(b[i].astype(np.float64), prec)}\n"
    print(out[:-1])

In [146]:
# Solve for square A
def solve_gauss(A,b, dtype=np.float64, prec=5):
    n = A.shape[0]
    assert A.shape[0] == A.shape[1], "A should be square"
    assert n == b.shape[0], f"A and b should have compatible sizes (A's {n} != b's {b.shape[0]})"

    # U for upper triangular form of A
    # v for modified b
    U = A.copy().astype(dtype)
    v = b.copy().astype(dtype)

    # Part A. Perform Gaussian elimination

    k = 0
    while k < n-1:
        print(f"\nGaussian elimination step {k}.")

        # 1. Find max element in a column under the k'th element of diagonal
        M, absM, Mi = U[k,k], np.abs(U[k,k]), k
        for i in range(k+1,n):
            a = np.abs(U[i,k])
            if a > absM:
                M, absM, Mi = U[i,k], a, i
        print(f"> pivot element is ({Mi},{Mi})")
        if Mi == k:
            print("> don't swap rows")
        else:
            print(f">  swap rows {Mi} and {k}")
        
        # 2.1. Swap k'th and i'th rows to make this element a pivot
        U[[k,Mi]] = U[[Mi,k]]
        # 2.2. Swap k'th and i'th elements of v
        v[[Mi,k]] = v[[k,Mi]]

        # if Mi != k:
        #     print("> swap the rows:")
        #     print_aug_mat(U,v,prec)

        # 3. Multiply current row by 1/M
        invM = dtype(1/M)
        print(f"> invM = {dtype}({invM})")

        U[k, k+1:] = U[k, k+1:] * invM
        U[k,k] = 1
        v[k] = v[k] * invM

        # 4. Subtract current row from all the next ones
        # - v[k+1:] is all elements of a vector after the k'th one
        # - U[k+1:, k+1:] is all elements to the right bottom of the pivot element
        # - U[k+1:, k] is a column below a pivot element
        # - U[k, k+1:] is a row to the right of a pivot element
        # - arr[:, None] makes a column vector out of a row vector

        # 4.1. First for vector v
        v[k+1:] = v[k+1:] - U[k+1:, k] * v[k]
        # 4.2. Then for matrix U
        # - do not compute elements in columns to the left of a pivot element since they should become 0
        U[k+1:, k+1:] = U[k+1:, k+1:] - U[k+1:, k][:, None] * U[k,k+1:]
        U[k+1:, k] = 0 
        
        print("> divide and subtract:")
        print_aug_mat(U,v,prec)

        k+=1
    
    # 5. Compute the last row
    v[-1] = v[-1] / U[-1,-1]
    U[-1,-1] = 1

    print("\nGaussian elimination result:")
    print_aug_mat(U,v,prec)

    # --------
    # Part B. Backpropagation

    # A[k,k:n] @ x[k:n] = b[k]
    # A[k,k] * x[k] + A[k,k+1:n] @ x[k+1:n] = b[k]
    # A[k,k] = 1;   x[k+1:n] is known
    # x[k] = b[k] - A[k,k+1:n] @ x[k+1:n]

    x = np.zeros(n, dtype=dtype)
    x[n-1] = v[n-1]
    # print(f"x[{n-1}] = v[{n-1}] = {x[-1]}")
    for k in range(n-2, -1, -1):
        x[k] = v[k] - U[k,k+1:n] @ x[k+1:n]
        # print(f"x[{k}] = {v[k]} - {U[k,k+1:n]} @ {x[k+1:n]} = {v[k]} -  {U[k,k+1:n] @ x[k+1:n]}")
    
    print("> x found:")
    print(x)

    return x, U, v


In [150]:
for ntask, (A, b) in enumerate(tasks):
    print(f"Task {ntask+1}, float16:")

    print("Initial A|b:")
    print_aug_mat(A,b, PREC)

    x,U,v = solve_gauss(A,b, np.float16, PREC)

    true_x = np.linalg.solve(A, b)
    print(f"True x* =", true_x.round(PREC))
    print(f"> sup|Ax-b| = {np.max(np.abs(A@x-b)):.4e}")
    print(f"> sup|x-x*| = {np.max(np.abs(x-true_x)):.4e}")
    print("=============================")
    print()
    
    print(f"Task {ntask+1}, float32:")

    print("Initial A|b:")
    print_aug_mat(A,b, PREC)

    x,U,v = solve_gauss(A,b, np.float32, PREC)

    true_x = np.linalg.solve(A, b)
    print(f"True x* =", true_x.round(PREC))
    print(f"> sup|Ax-b| = {np.max(np.abs(A@x-b)):.4e}")
    print(f"> sup|x-x*| = {np.max(np.abs(x-true_x)):.4e}")
    print("=============================")
    print()

Task 1, float16:
Initial A|b:
-3.0      -8.0      -3.0       | 5.0
3.0       -9.0      -9.0       | 7.0
2.0       -5.0      -1.0       | 0.0

Gaussian elimination step 0.
> pivot element is (0,0)
> don't swap rows
> invM = <class 'numpy.float16'>(-0.333251953125)
> divide and subtract:
1.0      2.66602   1.0        | -1.66602
0.0      -17.0     -12.0      | 12.0
0.0      -10.32812 -3.0       | 3.33203

Gaussian elimination step 1.
> pivot element is (1,1)
> don't swap rows
> invM = <class 'numpy.float16'>(-0.058837890625)
> divide and subtract:
1.0      2.66602  1.0       | -1.66602
0.0      1.0      0.70605   | -0.70605
0.0      0.0      4.29297   | -3.96094

Gaussian elimination result:
1.0      2.66602  1.0       | -1.66602
0.0      1.0      0.70605   | -0.70605
0.0      0.0      1.0       | -0.92285
> x found:
[-0.5977 -0.0547 -0.923 ]
True x* = [-0.59817 -0.05479 -0.92237]
> sup|Ax-b| = 4.8828e-03
> sup|x-x*| = 5.1727e-04

Task 1, float32:
Initial A|b:
-3.0      -8.0      -3.0    

## Задача 2
Знайти обернену матрицю для A.

Розглядаємо родину рівнянь $A\alpha_i = \hat e_i$. 
В результаті, $\alpha_i$ -- стануть стовпчиками оберненої матриці $A^{-1}$

Ці n рівнянь розв'язуємо методом Гауса

In [151]:
def inv_gauss(_A, dtype=np.float64):
    A = _A.astype(dtype)
    n = A.shape[0]
    assert A.shape[0] == A.shape[1], "A should be square"

    lu_factor = sp.linalg.lu_factor(A)
    AinvT = np.zeros((n,n), dtype=dtype)
    for k in range(n):
        e = np.zeros(n, dtype=dtype)
        e[k] = 1
        AinvT[k] = sp.linalg.lu_solve(lu_factor, e)
    
    return AinvT.T
    


In [161]:
for i,(A,_) in enumerate(tasks):
    print(f"Task {i+1}: float16")
    Ainv = inv_gauss(A, np.float16)
    E = A @ Ainv 
    print("> A * Ainv = ")
    print(E.round(4))
    print(f"> sup|A^-1 - true A^-1| = {np.max(np.abs(np.linalg.inv(A)-Ainv)):.5g}")
    print()

    print(f"Task {i+1}: float32")
    Ainv = inv_gauss(A, np.float32)
    E = A @ Ainv 
    print("> A * Ainv = ")
    print(E.round(8))
    print(f"> sup|A^-1 - true A^-1| = {np.max(np.abs(np.linalg.inv(A)-Ainv)):.8g}")
    print()

Task 1: float16
> A * Ainv = 
[[ 1.0000e+00  1.0000e-04  4.0000e-04]
 [-3.0000e-04  1.0003e+00  0.0000e+00]
 [-2.0000e-04 -0.0000e+00  1.0001e+00]]
> sup|A^-1 - true A^-1| = 4.9051e-05

Task 1: float32
> A * Ainv = 
[[ 9.99999950e-01 -0.00000000e+00 -3.00000000e-08]
 [ 6.00000000e-08  1.00000003e+00 -1.30000000e-07]
 [ 3.00000000e-08  3.00000000e-08  9.99999880e-01]]
> sup|A^-1 - true A^-1| = 2.3882683e-08

Task 2: float16
> A * Ainv = 
[[ 1.0001e+00 -1.0000e-04  6.0000e-04]
 [ 2.0000e-04  1.0006e+00  1.0000e-04]
 [-1.0000e-04 -4.0000e-04  9.9950e-01]]
> sup|A^-1 - true A^-1| = 7.4745e-05

Task 2: float32
> A * Ainv = 
[[ 9.99999990e-01 -1.00000000e-08  1.00000000e-08]
 [-1.20000000e-07  9.99999900e-01 -1.40000000e-07]
 [ 5.00000000e-08  6.00000000e-08  1.00000009e+00]]
> sup|A^-1 - true A^-1| = 1.7322957e-08



## Задача 3

Визначити число обумовленості матриці A



### Обґрунтування

Let
$B = A + C,\quad h = b + \eta,\quad y = x + z$,  
де $A,b,x$ вважаються заданими точно, а $C,\eta,z$ - похибка.

$$(A+C)(x+z) = b+\eta$$
$$z = A^{-1}\left(\eta-Cx-Cz\right)$$

Норми матриці задаються як
$$\|A\|_m = \sup_{\|x\|_m \ne 0} \frac{\|Ax\|_m}{\|x\|_m}, \quad m=1,2,\infty$$
І обчислюються як
$$\|A\|_1 = \max_{j} \sum_{i=1}^n|a_{ij}|; \quad \|A\|_\infty = \max_i \sum_{j=1}^n|a_{ij}|$$
$$\|A\|_2 = \max_j \sigma_i, \quad \sigma_i \text{ - singular value of }A$$
$$\|A\|_2 = \max_j \sqrt{\lambda_i(A^*A)}, \quad \lambda_i \text{ - eigenvalues of} A^*A$$

Тоді 
$$\|z\| \le \frac{\|A^{-1}\|\left(\|\eta\| + \|C\|\|x\|\right)}{1 - \|A^{-1}\|\|C\|}$$

Нехай $\operatorname{cond} A = \|A^{-1}\|\|A\|$.
Тоді
$$\delta(x) = \frac{\|z\|}{\|x\|} \le \frac{\operatorname{cond} A}{1-\operatorname{cond} A \cdot \delta(A)} \left(\delta(b) + \delta(A)\right)$$

In [179]:
# m=0 for inf. m=1 or m=2 for L1, L2.
def mnorm(A, m):
    assert m in [0,1,2], "Only m=1,2,0 supported. m=0 for inf."

    if m == 0:
        return np.max(np.sum(np.abs(A), axis=1))
    if m == 1:
        return np.max(np.sum(np.abs(A), axis=0))
    if m == 2:
        s = sp.linalg.svd(A, full_matrices=False, compute_uv=False)
        print("Singular vals: ", s.round(4))
        return s.max()
    
def cond(A, m):
    return mnorm(A, m) * mnorm(sp.linalg.inv(A), m)

In [180]:
for i, (A, b) in enumerate(tasks):
    M = [0,1,2]

    print(f"Task {i+1}: float16")
    for m in M:
        print(f"cond_{m} A = {cond(A.astype(np.float16),m):.6g}")
    print()
    
    print(f"Task {i+1}: float32")
    for m in M:
        print(f"cond_{m} A = {cond(A.astype(np.float32),m):.6g}")
    print()
    

Task 1: float16
cond_0 A = 8.43836
cond_1 A = 13.2603
Singular vals:  [15.7967  5.109   2.7136]
Singular vals:  [0.3685 0.1957 0.0633]
cond_2 A = 5.82135

Task 1: float32
cond_0 A = 8.43836
cond_1 A = 13.2603
Singular vals:  [15.7967  5.109   2.7136]
Singular vals:  [0.3685 0.1957 0.0633]
cond_2 A = 5.82135

Task 2: float16
cond_0 A = 7.92798
cond_1 A = 8.71146
Singular vals:  [14.2663 11.1009  2.2522]
Singular vals:  [0.444  0.0901 0.0701]
cond_2 A = 6.33436

Task 2: float32
cond_0 A = 7.93227
cond_1 A = 8.71742
Singular vals:  [14.2675 11.1026  2.2522]
Singular vals:  [0.444  0.0901 0.0701]
cond_2 A = 6.33505

